In [1]:
import pandas as pd

In [3]:
df1 = pd.read_csv('2023_data/df1.csv', encoding='euc-kr')

In [4]:
df1.head()

,성별,나이,이동인구(합),행정동명
0,F,20,39.07,서울특별시 종로구 사직동
1,F,20,*,서울특별시 종로구 사직동
2,F,20,*,서울특별시 종로구 사직동
3,F,25,*,서울특별시 종로구 사직동
4,F,25,4.08,서울특별시 종로구 사직동
